In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Cell 1 - Install dependencies (run once)
!pip install -q transformers datasets huggingface_hub accelerate sentencepiece
# bitsandbytes/quantization isn't required for inference-only but may be useful if you use a quantized model.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dep

In [16]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")  # your Hugging Face token stored in Kaggle secrets
os.environ["HF_TOKEN"] = hf_token


In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "O1-OPEN/OpenO1-Qwen-7B-v0.1"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto" if device=="cuda" else None,
    torch_dtype=torch.float16 if device=="cuda" else None,
    use_auth_token=hf_token
)
model.eval()


ValueError: Unrecognized model in O1-OPEN/OpenO1-Qwen-7B-v0.1. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v3, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, effic

In [4]:
# Cell 4 - Small helper functions: generation, normalize answers, scoring
import numpy as np

def generate_text(prompt, max_new_tokens=128, do_sample=False, temperature=0.0):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=do_sample, temperature=temperature)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # return only the generated suffix (after prompt)
    if prompt in text:
        return text.split(prompt, 1)[1].strip()
    return text.strip()

# math answer normalizer (simple)
def normalize_math_answer(text):
    # remove commas, words, extract first numeric-like token
    s = text.strip().lower()
    s = s.replace(",", "")
    # common prefixes "the answer is", "a:"
    s = re.sub(r"^(answer[:\s]*)", "", s)
    # try to find a number or fraction
    m = re.search(r"(-?\d+(\.\d+)?(/\d+)?)", s)
    if m:
        return m.group(1)
    # fallback: return cleaned text
    return re.sub(r"\s+", " ", s).strip()

# multiple-choice answer normalizer (for MMLU/Hellaswag/ARC etc.)
def normalize_choice(text):
    t = text.strip().upper()
    # prefer single letter A/B/C/D
    m = re.search(r"\b([A-D])\b", t)
    if m:
        return m.group(1)
    # maybe returns option text; we can't map automatically, return full text
    return t


In [5]:
max_examples = 100  # set to None for full dataset (may be large); use small for quick testing

output_file = "eval_results.json"
tasks_to_run = ["gsm8k", "math", "mmlu", "hellaswag", "arc_challenge", "bbh"]

In [10]:
# Cell 5 - Evaluate GSM8K (exact-match numeric)
from datasets import load_dataset
from tqdm.auto import tqdm
from pathlib import Path
import os, json, math, re
def eval_gsm8k(n=None):
    ds = load_dataset("gsm8k", "main", split="test")
    if n:
        ds = ds.select(range(min(n, len(ds))))
    correct = 0
    total = 0
    for ex in tqdm(ds, desc="GSM8K"):
        prompt = ex["question"] + "\n\nLet's think step by step.\nAnswer:"
        ans = generate_text(prompt, max_new_tokens=128, do_sample=False, temperature=0.0)
        pred = normalize_math_answer(ans)
        gold = normalize_math_answer(ex["answer"])
        total += 1
        if pred == gold:
            correct += 1
    return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}

# Quick run (use max_examples to limit)
gsm8k_res = eval_gsm8k(n=max_examples)
print("GSM8K:", gsm8k_res)


GSM8K:   0%|          | 0/100 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

GSM8K: {'accuracy': 0.14, 'correct': 14, 'total': 100}


In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm
from pathlib import Path
import os, json, math, re

# Cell 6 - Evaluate Swallow-MATH (exact match of final answer)
def eval_swallow_math(n=None):
    ds = load_dataset("tokyotech-llm/swallow-math", split="test")
    if n:
        ds = ds.select(range(min(n, len(ds))))
    correct=0; total=0
    for ex in tqdm(ds, desc="Swallow-MATH"):
        q = ex.get("question", "")
        a = ex.get("answer", "")

        # decode bytes if necessary
        if isinstance(q, (bytes, bytearray)):
            q = q.decode("utf-8", errors="ignore")
        if isinstance(a, (bytes, bytearray)):
            a = a.decode("utf-8", errors="ignore")

        prompt = q + "\n\nLet's think step by step.\nAnswer:"
        ans = generate_text(prompt, max_new_tokens=256, do_sample=False)
        pred = normalize_math_answer(ans)
        gold = normalize_math_answer(a)

        total += 1
        if pred == gold and gold!="":
            correct += 1
    return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}

# May fail if dataset missing/format mismatch. Use max_examples small to debug.
try:
    swallow_math_res = eval_swallow_math(n=max_examples)
    print("Swallow-MATH:", swallow_math_res)
except Exception as e:
    print("Swallow-MATH eval failed. Error:", e)
    swallow_math_res = {"error": str(e)}


In [23]:
from datasets import load_dataset
from tqdm.auto import tqdm
from pathlib import Path
import os, json, math, re
# Cell 6 - Evaluate MATH (exact match of final boxed answer)
# MATH dataset is large and has many long problems, use small n for debugging
def eval_math(n=None):
    ds = load_dataset("math_dataset", "algebra__linear_1d", split="test")  # dataset id may vary; fallback to 'math'
    # If the above fails, use datasets 'MATH' / check available config; handle exceptions
    # For safety below, try a few dataset names
    if len(ds)==0:
        ds = load_dataset("MATH", split="test")
    if n:
        ds = ds.select(range(min(n, len(ds))))
    correct=0; total=0
    for ex in tqdm(ds, desc="MATH"):
        prompt = ex["question"] + "\n\nLet's think step by step.\nAnswer:"
        ans = generate_text(prompt, max_new_tokens=256, do_sample=False)
        pred = normalize_math_answer(ans)
        # gold answer fields vary; try some keys
        gold = ex.get("answer") or ex.get("solution") or ""
        gold = normalize_math_answer(gold)
        total += 1
        if pred == gold and gold!="":
            correct += 1
    return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}

# May fail if dataset names differ. Use max_examples small to debug.
try:
    math_res = eval_math(n=max_examples)
    print("MATH:", math_res)
except Exception as e:
    print("MATH eval failed (dataset name or format mismatch). Error:", e)
    math_res = {"error": str(e)}


MATH:   0%|          | 0/100 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

MATH: {'accuracy': 0.01, 'correct': 1, 'total': 100}


In [ ]:
# Single cell: evaluate model on tokyotech-llm/swallow-math (zero-shot)
import os, re, json
import torch
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# CONFIG
model_name = "O1-OPEN/OpenO1-Qwen-7B-v0.1"   # change to your model if needed
dataset_id = "tokyotech-llm/swallow-math"
split = "test"
max_examples = 100     # set to None to run full split (may be slow / OOM)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model & tokenizer
print("Loading tokenizer and model (may take a while)...")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto" if device=="cuda" else None,
    torch_dtype=torch.float16 if device=="cuda" else None,
)
model.eval()
print("Model ready on device:", device)

# Helpers
def generate_text(prompt, max_new_tokens=128, do_sample=False, temperature=0.0):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=do_sample, temperature=temperature)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    if isinstance(prompt, str) and prompt.strip() and prompt in text:
        return text.split(prompt, 1)[1].strip()
    return text.strip()

def normalize_math_answer(text):
    if text is None:
        return ""
    if isinstance(text, (bytes, bytearray)):
        text = text.decode("utf-8", errors="ignore")
    text = str(text).strip().lower()
    text = re.sub(r"^(answer[:\s-]*|a[:\s-]*)", "", text)
    text = text.replace(",", "")
    m = re.search(r"-?\d+(\.\d+)?(/\d+)?", text)
    if m:
        return m.group(0)
    words_to_nums = {"zero":"0","one":"1","two":"2","three":"3","four":"4","five":"5","six":"6","seven":"7","eight":"8","nine":"9","ten":"10"}
    for w,n in words_to_nums.items():
        if re.search(r"\b"+w+r"\b", text):
            return n
    return re.sub(r"\s+"," ", text).strip()

# Load dataset
print(f"Loading dataset {dataset_id} split={split} ...")
ds = load_dataset(dataset_id, split=split)
print("Dataset loaded. Number of examples:", len(ds))

# Auto-detect probable question & answer fields (common names)
sample = next(iter(ds))
candidates_q = ["problem","question","input","prompt","text","body"]
candidates_a = ["answer","solution","target","output","label","answers"]

question_field = None
answer_field = None
for k in candidates_q:
    if k in sample:
        question_field = k
        break
for k in candidates_a:
    if k in sample:
        answer_field = k
        break

# fallback: pick longest field as question, shortest as answer
if question_field is None or answer_field is None:
    lengths = {k: (len(str(v)) if v is not None else 0) for k,v in sample.items()}
    sorted_fields = sorted(lengths.items(), key=lambda x: -x[1])
    if question_field is None:
        question_field = sorted_fields[0][0]
    if answer_field is None:
        # pick a different (shorter) field
        for k,l in sorted_fields[1:]:
            if k != question_field:
                answer_field = k
                break

print("Using fields -> question:", question_field, "  answer:", answer_field)

# Evaluation loop (decodes bytes, etc.)
def eval_swallow_math(ds, q_field, a_field, n=None):
    total = 0
    correct = 0
    subset = ds.select(range(min(n, len(ds)))) if n else ds
    for ex in tqdm(subset, desc="Swallow-MATH eval"):
        q = ex.get(q_field, "")
        a = ex.get(a_field, "")

        # decode bytes if necessary
        if isinstance(q, (bytes, bytearray)):
            q = q.decode("utf-8", errors="ignore")
        if isinstance(a, (bytes, bytearray)):
            a = a.decode("utf-8", errors="ignore")

        # flatten lists/dicts if present
        if isinstance(q, (list,tuple)):
            q = " ".join(map(str,q))
        if isinstance(q, dict):
            q = " ".join(map(str,q.values()))
        if isinstance(a, (list,tuple)):
            a = a[0] if len(a)>0 else ""
        if isinstance(a, dict):
            a = a.get("answer", list(a.values())[0] if len(a)>0 else "")

        prompt = str(q).strip() + "\n\nLet's think step by step.\nAnswer:"
        try:
            pred_text = generate_text(prompt, max_new_tokens=128, do_sample=False)
        except Exception as e:
            # if a single generation fails, skip example
            print("Generation error (skipping):", e)
            continue

        pred = normalize_math_answer(pred_text)
        gold = normalize_math_answer(a)

        total += 1
        if gold != "" and pred == gold:
            correct += 1

    return {"accuracy": correct/total if total>0 else 0.0, "correct": correct, "total": total}

# Run evaluation
res = eval_swallow_math(ds, question_field, answer_field, n=max_examples)
print("Swallow-MATH eval result:", res)

# Save results
outpath = "swallow_math_eval.json"
with open(outpath, "w") as f:
    json.dump(res, f, indent=2)
print("Saved results to", outpath)


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "O1-OPEN/OpenO1-LLama-8B-v0.1"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, use_auth_token=hf_token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [20]:
# Cell 7 - Evaluate multiple-choice tasks (MMLU, Hellaswag, ARC, BBH where applicable)
# We'll handle MMLU (57 tasks aggregated) using the 'mmlu' dataset from HF; some datasets may require processing.
# For BBH and others, dataset ids vary; we'll try standard names, otherwise leave as TODO.

def eval_multiple_choice(dataset_name, split="test", n=None, prompt_template=None, choice_key="choices", question_key="question", answer_key="answer"):
    ds = load_dataset(dataset_name, split=split)
    if n:
        ds = ds.select(range(min(n, len(ds))))
    correct=0; total=0
    for ex in tqdm(ds, desc=dataset_name):
        # Build a zero-shot prompt. Some datasets have different formats.
        q = ex.get(question_key) or ex.get("question") or ex.get("context") or ""
        choices = ex.get(choice_key) or ex.get("choices") or ex.get("options") or None
        if choices is None:
            # try alternative structures
            if "choices" in ex:
                choices = ex["choices"]
            else:
                choices = []
        # simple prompt
        prompt = q + "\n\nChoices:\n"
        if isinstance(choices, dict):
            # sometimes choices is a dict with "text" list
            choice_list = choices.get("text", [])
        else:
            choice_list = choices if isinstance(choices, list) else []
        for i, ch in enumerate(choice_list):
            prompt += f"{chr(65+i)}. {ch}\n"
        prompt += "\nAnswer (choose letter):"
        out = generate_text(prompt, max_new_tokens=30, do_sample=False)
        pred = normalize_choice(out)
        gold = ex.get(answer_key)
        # gold may be index or letter or text - try to normalize
        if isinstance(gold, int):
            gold_letter = chr(65+gold)
        elif isinstance(gold, str) and gold.strip().upper() in ["A","B","C","D"]:
            gold_letter = gold.strip().upper()
        else:
            # try to map gold text to a letter
            gold_letter = None
            # if gold is text, find matching choice
            for i,ch in enumerate(choice_list):
                if isinstance(gold,str) and gold.strip().lower() in str(ch).lower():
                    gold_letter = chr(65+i)
                    break
        total += 1
        if gold_letter and pred==gold_letter:
            correct += 1
    return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}

# Example: Hellaswag
try:
    hellaswag_res = eval_multiple_choice("hellaswag", split="validation", n=max_examples, question_key="context", choice_key="endings", answer_key="label")
    print("Hellaswag:", hellaswag_res)
except Exception as e:
    print("Hellaswag eval failed:", e)
    hellaswag_res = {"error": str(e)}


hellaswag:   0%|          | 0/100 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

Hellaswag: {'accuracy': 0.01, 'correct': 1, 'total': 100}


In [21]:
# Cell 8 - Try MMLU (use 'mmlu' dataset from HF if installed)
try:
    # MMLU has many subject subdatasets; to get full MMLU use the 'mmlu' dataset or process per subject
    mmlu_ds = load_dataset("cais/mmlu", "abstract_algebra", split="test")
    if max_examples:
        mmlu_ds = mmlu_ds.select(range(min(max_examples, len(mmlu_ds))))
    # MMLU examples often have 'question', 'options', 'answer'
    def run_mmlu(ds):
        correct=0; total=0
        for ex in tqdm(ds, desc="MMLU"):
            q = ex.get("input") or ex.get("question") or ex.get("prompt") or ""
            options = ex.get("options") or ex.get("choices") or ex.get("targets") or []
            # build prompt
            prompt = q + "\n\nChoices:\n"
            for i,opt in enumerate(options):
                prompt += f"{chr(65+i)}. {opt}\n"
            prompt += "\nAnswer (choose letter):"
            out = generate_text(prompt, max_new_tokens=20)
            pred = normalize_choice(out)
            gold = ex.get("output") or ex.get("answer")
            gold_letter = None
            if isinstance(gold, int):
                gold_letter = chr(65+gold)
            elif isinstance(gold, str) and gold.strip().upper() in ["A","B","C","D"]:
                gold_letter = gold.strip().upper()
            else:
                # attempt to map by text match
                for i,opt in enumerate(options):
                    if isinstance(gold,str) and gold.strip().lower() in str(opt).lower():
                        gold_letter = chr(65+i)
                        break
            total += 1
            if gold_letter and pred==gold_letter:
                correct += 1
        return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}

    mmlu_res = run_mmlu(mmlu_ds)
    print("MMLU (sample):", mmlu_res)
except Exception as e:
    print("MMLU eval failed:", e)
    mmlu_res = {"error": str(e)}


MMLU:   0%|          | 0/100 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

MMLU (sample): {'accuracy': 0.32, 'correct': 32, 'total': 100}


In [24]:
from datasets import load_dataset
from tqdm.auto import tqdm

# ---------- ARC-Challenge ----------
def eval_arc_challenge(n=None):
    ds = load_dataset("ai2_arc", "ARC-Challenge", split="test")
    if n:
        ds = ds.select(range(min(n, len(ds))))
    correct, total = 0, 0
    for ex in tqdm(ds, desc="ARC-Challenge"):
        q = ex.get("question", "")
        choices = ex.get("choices", {}).get("text", [])
        ans_key = ex.get("answerKey", "")

        # Build multiple-choice style prompt
        choice_str = "\n".join([f"{chr(65+i)}. {c}" for i, c in enumerate(choices)])
        prompt = q + "\n" + choice_str + "\n\nAnswer:"
        
        model_out = generate_text(prompt, max_new_tokens=64, do_sample=False)
        pred = model_out.strip().upper()[:1]  # first char like "A","B","C"
        
        total += 1
        if pred == ans_key:
            correct += 1
    return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}


# ---------- BBH (boolean_expressions subset) ----------
def eval_bbh_boolean(n=None):
    ds = load_dataset("lukaemon/bbh", "boolean_expressions", split="test")
    if n:
        ds = ds.select(range(min(n, len(ds))))
    correct, total = 0, 0
    for ex in tqdm(ds, desc="BBH-Boolean"):
        q = ex.get("input", "")
        gold = ex.get("target", "")

        prompt = q + "\n\nAnswer:"
        ans = generate_text(prompt, max_new_tokens=128, do_sample=False)
        pred = ans.strip().lower()
        gold = gold.strip().lower()

        total += 1
        if pred == gold:
            correct += 1
    return {"accuracy": correct/total if total>0 else 0, "correct": correct, "total": total}


# ---------- Run them ----------
try:
    arc_res = eval_arc_challenge(n=max_examples)
    print("ARC-Challenge:", arc_res)
except Exception as e:
    print("ARC-Challenge eval failed:", e)
    arc_res = {"error": str(e)}

try:
    bbh_res = eval_bbh_boolean(n=max_examples)
    print("BBH-Boolean:", bbh_res)
except Exception as e:
    print("BBH-Boolean eval failed:", e)
    bbh_res = {"error": str(e)}


README.md: 0.00B [00:00, ?B/s]

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

ARC-Challenge/test-00000-of-00001.parque(…):   0%|          | 0.00/204k [00:00<?, ?B/s]

ARC-Challenge/validation-00000-of-00001.(…):   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

...

boolean_expressions/test-00000-of-00001.(…):   0%|          | 0.00/4.52k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

BBH-Boolean:   0%|          | 0/100 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

BBH-Boolean: {'accuracy': 0.0, 'correct': 0, 'total': 100}


In [ ]:
# Cell 9 - ARC-Challenge and BBH attempt (may require dataset ids and preprocessing)
try:
    arc_res = eval_multiple_choice("ai2_arc", split="ARC-Challenge", n=max_examples, question_key="question", choice_key="choices", answer_key="answerKey")
    print("ARC-Challenge:", arc_res)
except Exception as e:
    print("ARC eval failed:", e)
    arc_res = {"error": str(e)}

# BBH is part of BIG-Bench hard. HF dataset name might be 'bbh' or must be prepared.
try:
    bbh_res = eval_multiple_choice("bbh", split="test", n=max_examples, question_key="question", choice_key="choices", answer_key="answer")
    print("BBH:", bbh_res)
except Exception as e:
    print("BBH eval failed (dataset naming/format likely):", e)
    bbh_res = {"error": str(e)}
